In [36]:
import validation
import importlib
importlib.reload(validation)
from validation import *
from etl_eda import *


In [25]:
data = load('data.csv')

In [37]:
validator = Validator(data)
if validator.validate():
    print("Validation is passed")
else:
    print("Validation is failed")

All values in column item_category_id are numeric
All values in column 'item_category_id' are in range: [0, 83].
All values in column main_category_id are numeric
All values in column 'main_category_id' are in range: [0, 19].
All values in column minor_category_id are numeric
All values in column 'minor_category_id' are in range: [0, 66].
All values in column item_id are numeric
All values in column 'item_id' are in range: [0, 22169].
All values in column shop_id_encoded are numeric
All values in column 'shop_id_encoded' are in range: [0, 57].
All values in column city_id are numeric
All values in column 'city_id' are in range: [0, 29].
All values in column date_block_num are numeric
All values in column 'date_block_num' are in range: [0, 34].
All values in column revenue_log are numeric
All values in column revenue_log are positive
All values in column item_price_log are numeric
All values in column item_price_log are positive
All values in column month are numeric
All values in colum